In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/tijes/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

## Define Search

To allow us to easily perform different searches in the future, we will define variables for LOCATION and TERM set for our particular search conditions. Then, when we want to use a different location or term, we can just redefine these variables. This streamlines the code and makes it more readable and reproducible.  

In [3]:
# set our API call parameters and filename before the first call
LOCATION = 'Baltimore, MD,21202'
TERM = 'Crab Cakes'

## Create a results-in-progress JSON file, but only if it doesn't exist.

This is the file where your results will be saved.  Note: you must rename your JSON_FILE for different queries to prevent confusing results from other searches. We recommend you include your search terms in the filename. 

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = f"Data/results_in_progress_Crab_cakes.json"
JSON_FILE

'Data/results_in_progress_Crab_cakes.json'

Check if our JSON_FILE already exists. This will prevent us from accidently overwriting an existing file.

If it doesn't exist:

- Create any folders needed for the file path.
- Save an empty list as JSON_File

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")



[i] Data/results_in_progress_Crab_cakes.json already exists.


## Determine how many results are already in the file

Load the results file to determine the # of results we have previously retrieved. If you just created the file, you would expect it to be empty.  

We will use this as our offset parameter for our API call.  Even if this is your first API call, and the number is 0, we want to define "n_results" based on the length of "previous_results." 

In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 424 previous results found.


## Figure out how many pages of results we will need

We will perform our first query to get our first page of results and the total number of results.  We will then (via code) calculate how many pages we will need to retrieve all of our results.

In [7]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results+1)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
## How many results total?
total_results = results['total']
total_results

433

In [9]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

0

There are over 400 businesses to retrieve from our API and we can get 20 results at a time (per "page").

- We can calculate the # of results remaining by subtracting our offset (length of our previous results) from our total.

- Then we can determining how many pages we will need by dividing the results by 20 (or whatever the value happens to be for results per page)

- Note that we need to round up the number of pages in order to get all of the results. Even if there is only 1 result on the last page, we want to include that page! To do this we will use math.ceil. 

In [10]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

ZeroDivisionError: division by zero

When this example was written, there were 437 results and 20 results per page.  437 /20 = 21.85. Rounding UP gives us 22 pages.  We expect the first 21 pages to have 20 results and the last page to have the final 17 results.  Notice that we have assigned the number of pages as n_pages here. We will use this value in our next segment of code.

You can see that having to manually go through 22 pages would be quite time consuming and inefficient! First we are going to save the first page into our file, and then we will add on to it with a for loop.

## Add this page of results to .json file

Our API returns our results in JSON format, with the businesses in a list of dictionaries. We will append the first page of businesses to our previous_results (which is very likely empty) and then save to disk.

In [ ]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


## Set up a progress bar in our for loop.

To keep us informed about where we are in our loop, we will add a progress bar to our for loop.

TQDM is a package designed for adding animated progress bars to Python processes.  It is not currently included in your dojo-env, so you are going to install it manually by opening a new Terminal/GitBash window and running the following command:

TQDM  is easy to use and simply needs to know what we are looping through. If you wanted to test tqdm in action, but your loop is too fast, you can import time and use time.sleep to add a pause within your for loop. We will also use time.sleep when executing many API calls so that we do not overwhelm the server.

In [ ]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

## For Loop to call each page

The loop below will iterate through each page of the results by starting at the appropriate offset.  It will then append the results to the previous_results.  This may take some time, so check out the progress bar!

In [ ]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results+1)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

# After the loop has finished

## Convert .json to dataframe

Load in the "results in progress" JSON file into a DataFrame:

In [ ]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


## Check for duplicates
Check IDs to make sure you have unique results.

In [ ]:
# check for duplicate IDs
final_df.duplicated(subset='id').sum()

In [ ]:
# Drop the duplicates
final_df.drop_duplicates(inplace=True)

## Save the final DataFrame to a .csv (or a .csv.gz if its too big for the GitHub file size limit).

In [11]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_crab_cakes.csv.gz', compression='gzip',index=False)


NameError: name 'final_df' is not defined